# spacetrackのテスト その1 (gp API編)

gp系APIで取得できるデータの項目: https://www.space-track.org/basicspacedata/modeldef/class/gp/format/html

(formatにtle, 3leを指定した場合を除いて) Orbit Mean-Elements Message (OMM) に準拠した項目名になっている。

tle系APIとは、取得できるデータの項目や内容が若干異なる。
APOGEE が APOAPSIS に、PERIGEE が PERIAPSIS、DECAYED が DECAY_DATE に変わったり、COUNTRY_CODE, LAUNCH_DATE などが追加される。

https://www.space-track.org/documentation#faq によると、現在では tle, tle_latest, tle_publish API は非推奨となっており、今後は gp, gp_history の利用が推奨される。

In [1]:
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pandas as pd
import json
import time

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 80)

In [3]:
# userid and password are set in here
import spacetrackaccount
# print("Login with " + spacetrackaccount.userid)

In [4]:
st = SpaceTrackClient(spacetrackaccount.userid, spacetrackaccount.password)

In [5]:
def mycallback(until):
    duration = int(round(until - time.monotonic()))
    print('Sleeping for {:d} seconds.'.format(duration))

st.callback = mycallback

##  全衛星の最新データをいろいろな形式でダウンロードする
TLE, 3LE, CSV, OMM Standard XML, OMM Standard KVN, JSON (detault), HTML

In [6]:
# TLE
data_tle = st.gp(epoch='>now-30', orderby='norad_cat_id', format='tle')
with open('download/gp_latest_30day.tle', 'w') as fp:
    fp.write(data_tle)

In [7]:
# 3LE
data_3le = st.gp(epoch='>now-30', orderby='norad_cat_id', format='3le')
with open('download/gp_latest_30day.3le', 'w') as fp:
    fp.write(data_3le)

In [8]:
# CSV
data_csv = st.gp(epoch='>now-30', orderby='norad_cat_id', format='csv')
with open('download/gp_latest_30day.csv', 'w') as fp:
    fp.write(data_csv)

In [9]:
# OMM Standard XML
data_xml = st.gp(epoch='>now-30', orderby='norad_cat_id', format='xml')
with open('download/gp_latest_30day.xml', 'w') as fp:
    fp.write(data_xml)

In [10]:
# OMM Standard KVN
data_kvn = st.gp(epoch='>now-30', orderby='norad_cat_id', format='kvn')
with open('download/gp_latest_30day.kvn', 'w') as fp:
    fp.write(data_kvn)

In [11]:
# JSON (default)
data_json = st.gp(epoch='>now-30', orderby='norad_cat_id', format='json')
with open('download/gp_latest_30day.json', 'w') as fp:
    fp.write(data_json)

In [12]:
# HTML
data_html = st.gp(epoch='>now-30', orderby='norad_cat_id', format='html')
with open('download/gp_latest_30day.html', 'w') as fp:
    fp.write(data_html)

### 各形式のサイズを比較
XMLのサイズ増加が著しい (tle APIと較べると約1.5倍)

In [13]:
# サイズを比較
print('TLE:\t{}\n3LE:\t{}\nCSV:\t{}\nXML:\t{}\nKVN:\t{}\nJSON:\t{}\nHTML:\t{}'
      .format(len(data_tle), len(data_3le), len(data_csv), len(data_xml), len(data_kvn), len(data_json), len(data_html)))

TLE:	2801260
3LE:	3115082
CSV:	11604095
XML:	37265183
KVN:	46100736
JSON:	22738242
HTML:	16626824


### データの中身を確認

- XMLの構造が複雑になっている
- XMLとKVNはTLE_LINE0, TLE_LINE1, TLE_LINE2 を含まない

In [14]:
# TLEの最初の6行を表示
for line in data_tle.splitlines()[0:6]:
    print(line)

1     5U 58002B   20308.28026105 -.00000002  00000-0 -27303-4 0  9990
2     5  34.2561 160.5895 1847138 257.1165  81.7729 10.84868808220426
1    11U 59001A   20308.30570535  .00000229  00000-0  11110-3 0  9998
2    11  32.8740 254.0817 1466665 315.3162  33.7976 11.85684888633206
1    12U 59001B   20307.82218806 +.00000365 +00000-0 +22198-3 0  9994
2    12 032.8976 226.8616 1666373 083.4083 295.2617 11.44346712295831


In [15]:
# 3LEの最初の9行を表示
for line in data_3le.splitlines()[0:9]:
    print(line)

0 VANGUARD 1
1     5U 58002B   20308.28026105 -.00000002  00000-0 -27303-4 0  9990
2     5  34.2561 160.5895 1847138 257.1165  81.7729 10.84868808220426
0 VANGUARD 2
1    11U 59001A   20308.30570535  .00000229  00000-0  11110-3 0  9998
2    11  32.8740 254.0817 1466665 315.3162  33.7976 11.85684888633206
0 VANGUARD R/B
1    12U 59001B   20307.82218806 +.00000365 +00000-0 +22198-3 0  9994
2    12 032.8976 226.8616 1666373 083.4083 295.2617 11.44346712295831


In [16]:
# CSVの最初の4行を表示
for line in data_csv.splitlines()[0:4]:
    print(line)

CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
"2.0","GENERATED VIA SPACE-TRACK.ORG API","2020-11-03T14:26:11","18 SPCS","VANGUARD 1","1958-002B","EARTH","TEME","UTC","SGP4","2020-11-03T06:43:34.554720","10.84868808","0.18471380","34.2561","160.5895","257.1165","81.7729","0","U","5","999","22042","-0.00002730300000","-0.00000002","0.0000000000000","8619.532","132.734","3833.543","649.250","PAYLOAD","MEDIUM","US","1958-03-17","AFETR","","2869770","164769082","0 VANGUARD 1","1     5U 58002B   20308.28026105 -.00000002  00000-0 -27303-4 0  9990","2     5  34.2561 160.5895

In [17]:
# XMLを整形して最初の60行を表示
import xml.etree.ElementTree as et
import xml.dom.minidom as md
for line in  md.parseString(et.tostring(et.fromstring(data_xml), 'utf-8')).toprettyxml(indent="    ").splitlines()[0:60]:
    print(line)

<?xml version="1.0" ?>
<ndm xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sanaregistry.org/r/ndmxml/ndmxml-1.0-master.xsd">
    <omm id="CCSDS_OMM_VERS" version="2.0">
        <header>
            <COMMENT>GENERATED VIA SPACE-TRACK.ORG API</COMMENT>
            <CREATION_DATE>2020-11-03T14:26:11</CREATION_DATE>
            <ORIGINATOR>18 SPCS</ORIGINATOR>
        </header>
        <body>
            <segment>
                <metadata>
                    <OBJECT_NAME>VANGUARD 1</OBJECT_NAME>
                    <OBJECT_ID>1958-002B</OBJECT_ID>
                    <CENTER_NAME>EARTH</CENTER_NAME>
                    <REF_FRAME>TEME</REF_FRAME>
                    <TIME_SYSTEM>UTC</TIME_SYSTEM>
                    <MEAN_ELEMENT_THEORY>SGP4</MEAN_ELEMENT_THEORY>
                </metadata>
                <data>
                    <meanElements>
                        <EPOCH>2020-11-03T06:43:34.554720</EPOCH>
                        <MEAN_M

In [18]:
# KVNの最初の40行を表示
for line in data_kvn.splitlines()[0:40]:
    print(line)

CCSDS_OMM_VERS                     =2.0                      
COMMENT                            =GENERATED VIA SPACE-TRACK.ORG API
CREATION_DATE                      =2020-11-03T14:26:11      
ORIGINATOR                         =18 SPCS                  
OBJECT_NAME                        =VANGUARD 1               
OBJECT_ID                          =1958-002B                
CENTER_NAME                        =EARTH                    
REF_FRAME                          =TEME                     
TIME_SYSTEM                        =UTC                      
MEAN_ELEMENT_THEORY                =SGP4                     
EPOCH                              =2020-11-03T06:43:34.554720
MEAN_MOTION                        =10.84868808              
ECCENTRICITY                       =0.18471380               
INCLINATION                        =34.2561                  
RA_OF_ASC_NODE                     =160.5895                 
ARG_OF_PERICENTER                  =257.1165                 

In [19]:
# JSONを整形して最初の46行を表示
for line in json.dumps(json.loads(data_json), indent=4).splitlines()[0:46]:
    print(line)

[
    {
        "CCSDS_OMM_VERS": "2.0",
        "COMMENT": "GENERATED VIA SPACE-TRACK.ORG API",
        "CREATION_DATE": "2020-11-03T14:26:11",
        "ORIGINATOR": "18 SPCS",
        "OBJECT_NAME": "VANGUARD 1",
        "OBJECT_ID": "1958-002B",
        "CENTER_NAME": "EARTH",
        "REF_FRAME": "TEME",
        "TIME_SYSTEM": "UTC",
        "MEAN_ELEMENT_THEORY": "SGP4",
        "EPOCH": "2020-11-03T06:43:34.554720",
        "MEAN_MOTION": "10.84868808",
        "ECCENTRICITY": "0.18471380",
        "INCLINATION": "34.2561",
        "RA_OF_ASC_NODE": "160.5895",
        "ARG_OF_PERICENTER": "257.1165",
        "MEAN_ANOMALY": "81.7729",
        "EPHEMERIS_TYPE": "0",
        "CLASSIFICATION_TYPE": "U",
        "NORAD_CAT_ID": "5",
        "ELEMENT_SET_NO": "999",
        "REV_AT_EPOCH": "22042",
        "BSTAR": "-0.00002730300000",
        "MEAN_MOTION_DOT": "-0.00000002",
        "MEAN_MOTION_DDOT": "0.0000000000000",
        "SEMIMAJOR_AXIS": "8619.532",
        "PERIOD": "132.

In [20]:
# HTMLを整形して表示
from IPython.display import display, HTML
display(HTML(data_html[0:data_html.find('</tr>', 3000)+5]))

CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:26:11,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-11-03T06:43:34.554720,10.84868808,0.18471380,34.2561,160.5895,257.1165,81.7729,0,U,5,999,22042,-0.00002730300000,-0.00000002,0.0000000000000,8619.532,132.734,3833.543,649.250,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,,2869770,164769082,0 VANGUARD 1,1 5U 58002B 20308.28026105 -.00000002 00000-0 -27303-4 0 9990,2 5 34.2561 160.5895 1847138 257.1165 81.7729 10.84868808220426
2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:36:10,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-11-03T07:20:12.942240,11.85684888,0.14666650,32.8740,254.0817,315.3162,33.7976,0,U,11,999,63320,0.00011110000000,0.00000229,0.0000000000000,8123.733,121.448,2937.077,554.118,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,,2869865,164771164,0 VANGUARD 2,1 11U 59001A 20308.30570535 .00000229 00000-0 11110-3 0 9998,2 11 32.8740 254.0817 1466665 315.3162 33.7976 11.85684888633206
2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T01:26:30,18 SPCS,VANGUARD R/B,1959-001B,EARTH,TEME,UTC,SGP4,2020-11-02T19:43:57.048384,11.44346712,0.16663730,32.8976,226.8616,83.4083,295.2617,0,U,12,999,29583,0.00022198000000,0.00000365,0.0000000000000,8318.214,125.835,3326.203,553.954,ROCKET BODY,MEDIUM,US,1959-02-17,AFETR,,2869318,164737888,0 VANGUARD R/B,1 12U 59001B 20307.82218806 +.00000365 +00000-0 +22198-3 0 9994,2 12 032.8976 226.8616 1666373 083.4083 295.2617 11.44346712295831


### DataFrameに変換する
データの型の自動変換や、欠損値の扱いに注意。

pandas.read_json や pandas.read_csv を用いる際には https://www.space-track.org/basicspacedata/modeldef/class/tle/format/html を参照の上、dtype を指定する方がよい。(以下の例ではdtypeは指定していない)

OBJECT_ID, RCS_SIZE, COUNTRY_CODE, LAUNCH_DATE, SITE, DECAY_DATE, DECAYD についてはデータが入っていないことがある。

以下、dtype を指定せず型を自動判定させる場合の扱い。

JSON では欠損値は null となっている。
pandas.read_json の結果、これらはNoneに変換される。(もし数値型のcolumnの場合にはNaNに変換されるが、gp APIを用いた場合はそのような列はない)

CSV では欠損値は単に長さ0の文字列となっている。
pandas.read_csv の結果、これらはいずれもNaNに変換される。

XML では欠損値は \<USER_DEFINED parameter="DECAYED"/\> のような空要素になっている。
変換方法によってどう扱われるかは異なる。
単純にDICT経由で変換する場合には長さ0の文字列になる。

In [21]:
# CSV
df_csv = pd.read_csv('download/gp_latest_30day.csv')
df_csv.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:26:11,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-11-03T06:43:34.554720,10.848688,0.184714,34.2561,160.5895,257.1165,81.7729,0,U,5,999,22042,-0.000027,-2.000000e-08,0.0,8619.532,132.734,3833.543,649.250,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,NaN,2869770,164769082,0 VANGUARD 1,1 5U 58002B 20308.28026105 -.00000002 00000-0 -27303-4 0 9990,2 5 34.2561 160.5895 1847138 257.1165 81.7729 10.84868808220426
1,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:36:10,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-11-03T07:20:12.942240,11.856849,0.146667,32.8740,254.0817,315.3162,33.7976,0,U,11,999,63320,0.000111,2.290000e-06,0.0,8123.733,121.448,2937.077,554.118,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,NaN,2869865,164771164,0 VANGUARD 2,1 11U 59001A 20308.30570535 .00000229 00000-0 11110-3 0 9998,2 11 32.8740 254.0817 1466665 315.3162 33.7976 11.85684888633206
2,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T01:26:30,18 SPCS,VANGUARD R/B,1959-001B,EARTH,TEME,UTC,SGP4,2020-11-02T19:43:57.048384,11.443467,0.166637,32.8976,226.8616,83.4083,295.2617,0,U,12,999,29583,0.000222,3.650000e-06,0.0,8318.214,125.835,3326.203,553.954,ROCKET BODY,MEDIUM,US,1959-02-17,AFETR,NaN,2869318,164737888,0 VANGUARD R/B,1 12U 59001B 20307.82218806 +.00000365 +00000-0 +22198-3 0 9994,2 12 032.8976 226.8616 1666373 083.4083 295.2617 11.44346712295831


In [22]:
# XML
# gp APIで取得するXMLは自動変換が難しい
import xml.etree.ElementTree as ET
import xmljson
root = ET.fromstring(data_xml)
data_dict = xmljson.yahoo.data(root)
df_xml_tmp = pd.json_normalize(data_dict['ndm']['omm'])
df_xml_tmp.head(3)

,id,version,header.COMMENT,header.CREATION_DATE,header.ORIGINATOR,body.segment.metadata.OBJECT_NAME,body.segment.metadata.OBJECT_ID,body.segment.metadata.CENTER_NAME,body.segment.metadata.REF_FRAME,body.segment.metadata.TIME_SYSTEM,body.segment.metadata.MEAN_ELEMENT_THEORY,body.segment.data.meanElements.EPOCH,body.segment.data.meanElements.MEAN_MOTION,body.segment.data.meanElements.ECCENTRICITY,body.segment.data.meanElements.INCLINATION,body.segment.data.meanElements.RA_OF_ASC_NODE,body.segment.data.meanElements.ARG_OF_PERICENTER,body.segment.data.meanElements.MEAN_ANOMALY,body.segment.data.tleParameters.EPHEMERIS_TYPE,body.segment.data.tleParameters.CLASSIFICATION_TYPE,body.segment.data.tleParameters.NORAD_CAT_ID,body.segment.data.tleParameters.ELEMENT_SET_NO,body.segment.data.tleParameters.REV_AT_EPOCH,body.segment.data.tleParameters.BSTAR,body.segment.data.tleParameters.MEAN_MOTION_DOT,body.segment.data.tleParameters.MEAN_MOTION_DDOT,body.segment.data.userDefinedParameters.USER_DEFINED
0,CCSDS_OMM_VERS,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:26:11,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-11-03T06:43:34.554720,10.84868808,0.18471380,34.2561,160.5895,257.1165,81.7729,0,U,5,999,22042,-0.00002730300000,-0.00000002,0.0000000000000,"[{'parameter': 'SEMIMAJOR_AXIS', 'content': '8619.532'}, {'parameter': 'PERI..."
1,CCSDS_OMM_VERS,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:36:10,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-11-03T07:20:12.942240,11.85684888,0.14666650,32.8740,254.0817,315.3162,33.7976,0,U,11,999,63320,0.00011110000000,0.00000229,0.0000000000000,"[{'parameter': 'SEMIMAJOR_AXIS', 'content': '8123.733'}, {'parameter': 'PERI..."
2,CCSDS_OMM_VERS,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T01:26:30,18 SPCS,VANGUARD R/B,1959-001B,EARTH,TEME,UTC,SGP4,2020-11-02T19:43:57.048384,11.44346712,0.16663730,32.8976,226.8616,83.4083,295.2617,0,U,12,999,29583,0.00022198000000,0.00000365,0.0000000000000,"[{'parameter': 'SEMIMAJOR_AXIS', 'content': '8318.214'}, {'parameter': 'PERI..."


In [23]:
df_xml_tmp['body.segment.data.userDefinedParameters.USER_DEFINED'][0]

[OrderedDict([('parameter', 'SEMIMAJOR_AXIS'), ('content', '8619.532')]),
 OrderedDict([('parameter', 'PERIOD'), ('content', '132.734')]),
 OrderedDict([('parameter', 'APOAPSIS'), ('content', '3833.543')]),
 OrderedDict([('parameter', 'PERIAPSIS'), ('content', '649.250')]),
 OrderedDict([('parameter', 'OBJECT_TYPE'), ('content', 'PAYLOAD')]),
 OrderedDict([('parameter', 'RCS_SIZE'), ('content', 'MEDIUM')]),
 OrderedDict([('parameter', 'COUNTRY_CODE'), ('content', 'US')]),
 OrderedDict([('parameter', 'LAUNCH_DATE'), ('content', '1958-03-17')]),
 OrderedDict([('parameter', 'SITE'), ('content', 'AFETR')]),
 OrderedDict([('parameter', 'DECAY_DATE')]),
 OrderedDict([('parameter', 'DECAYED')]),
 OrderedDict([('parameter', 'FILE'), ('content', '2869770')]),
 OrderedDict([('parameter', 'GP_ID'), ('content', '164769082')])]

In [24]:
# 再帰的に検索し、1つずつ追加する
import xml.etree.ElementTree as ET
root = ET.fromstring(data_xml)
all_records = []
for child in root:
    record = {}
    for element in child.iter():
        if element.tag in ['header', 'body', 'segment', 'metadata', 'data', 'meanElements', 'tleParameters', 'userDefinedParameters']:
            continue
        elif element.tag == 'omm':
            record[element.attrib['id']] = element.attrib['version']
        elif element.tag == 'USER_DEFINED':
            record[element.attrib['parameter']] = element.text
        else:
            record[element.tag] = element.text
    all_records.append(record)
df_xml = pd.DataFrame(all_records)
df_xml.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,DECAYED,FILE,GP_ID
0,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:26:11,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-11-03T06:43:34.554720,10.84868808,0.18471380,34.2561,160.5895,257.1165,81.7729,0,U,5,999,22042,-0.00002730300000,-0.00000002,0.0000000000000,8619.532,132.734,3833.543,649.250,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,None,None,2869770,164769082
1,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:36:10,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-11-03T07:20:12.942240,11.85684888,0.14666650,32.8740,254.0817,315.3162,33.7976,0,U,11,999,63320,0.00011110000000,0.00000229,0.0000000000000,8123.733,121.448,2937.077,554.118,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,None,None,2869865,164771164
2,2.0,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T01:26:30,18 SPCS,VANGUARD R/B,1959-001B,EARTH,TEME,UTC,SGP4,2020-11-02T19:43:57.048384,11.44346712,0.16663730,32.8976,226.8616,83.4083,295.2617,0,U,12,999,29583,0.00022198000000,0.00000365,0.0000000000000,8318.214,125.835,3326.203,553.954,ROCKET BODY,MEDIUM,US,1959-02-17,AFETR,None,None,2869318,164737888


In [25]:
# JSON
df_json = pd.read_json('download/gp_latest_30day.json')
# df_json = pd.read_json(data_json) でも可
df_json.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:26:11,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-11-03T06:43:34.554720,10.848688,0.184714,34.2561,160.5895,257.1165,81.7729,0,U,5,999,22042,-0.000027,-2.000000e-08,0.0,8619.532,132.734,3833.543,649.250,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,None,2869770,164769082,0 VANGUARD 1,1 5U 58002B 20308.28026105 -.00000002 00000-0 -27303-4 0 9990,2 5 34.2561 160.5895 1847138 257.1165 81.7729 10.84868808220426
1,2,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T14:36:10,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-11-03T07:20:12.942240,11.856849,0.146667,32.8740,254.0817,315.3162,33.7976,0,U,11,999,63320,0.000111,2.290000e-06,0.0,8123.733,121.448,2937.077,554.118,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,None,2869865,164771164,0 VANGUARD 2,1 11U 59001A 20308.30570535 .00000229 00000-0 11110-3 0 9998,2 11 32.8740 254.0817 1466665 315.3162 33.7976 11.85684888633206
2,2,GENERATED VIA SPACE-TRACK.ORG API,2020-11-03T01:26:30,18 SPCS,VANGUARD R/B,1959-001B,EARTH,TEME,UTC,SGP4,2020-11-02T19:43:57.048384,11.443467,0.166637,32.8976,226.8616,83.4083,295.2617,0,U,12,999,29583,0.000222,3.650000e-06,0.0,8318.214,125.835,3326.203,553.954,ROCKET BODY,MEDIUM,US,1959-02-17,AFETR,None,2869318,164737888,0 VANGUARD R/B,1 12U 59001B 20307.82218806 +.00000365 +00000-0 +22198-3 0 9994,2 12 032.8976 226.8616 1666373 083.4083 295.2617 11.44346712295831


## 特定の衛星の過去の全データを取得する

In [26]:
# 特定の衛星の過去の全データ
data = st.gp_history(norad_cat_id=25544, orderby='epoch', format='json')

In [27]:
with open('download/gp_history_25544_all.json', 'w') as fp:
    fp.write(data)

In [28]:
df = pd.read_json(data)

In [29]:
df.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-17T15:22:27,18 SPCS,ISS (ZARYA),1998-067A,EARTH,TEME,UTC,SGP4,1998-11-20T06:49:59.999807,16.050648,0.012536,51.5908,168.3788,86.4185,359.7454,0,U,25544,999,0,0.000000,-0.000037,0.000012,6638.557,89.716,343.644,177.200,PAYLOAD,LARGE,ISS,1998-11-20,TTMTR,NaN,35498,28983777,0 ISS (ZARYA),1 25544U 98067A 98324.28472222 -.00003657 11563-4 00000+0 0 9996,2 25544 051.5908 168.3788 0125362 086.4185 359.7454 16.05064833 05
1,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-17T15:22:27,18 SPCS,ISS (ZARYA),1998-067A,EARTH,TEME,UTC,SGP4,1998-11-20T07:58:35.072831,16.049718,0.012341,51.6173,168.1099,88.0187,273.4932,0,U,25544,999,1,0.005735,0.118396,0.000012,6638.814,89.721,342.608,178.749,PAYLOAD,LARGE,ISS,1998-11-20,TTMTR,NaN,35498,28983778,0 ISS (ZARYA),1 25544U 98067A 98324.33235038 .11839616 11568-4 57349-2 0 9993,2 25544 051.6173 168.1099 0123410 088.0187 273.4932 16.04971811 11
2,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-17T15:22:27,18 SPCS,ISS (ZARYA),1998-067A,EARTH,TEME,UTC,SGP4,1998-11-20T10:57:42.787008,16.051344,0.012586,51.5914,167.4317,91.3429,269.4598,0,U,25544,999,3,-0.000018,-0.000433,0.000012,6638.365,89.712,343.779,176.681,PAYLOAD,LARGE,ISS,1998-11-20,TTMTR,NaN,35498,28983779,0 ISS (ZARYA),1 25544U 98067A 98324.45674522 -.00043259 11566-4 -18040-4 0 9996,2 25544 051.5914 167.4317 0125858 091.3429 269.4598 16.05134416 30


In [30]:
# 従来のTLEの文字列も含まれている (XML, KVNには含まれない)
print(df.TLE_LINE0[0])
print(df.TLE_LINE1[0])
print(df.TLE_LINE2[0])

0 ISS (ZARYA)
1 25544U 98067A   98324.28472222 -.00003657  11563-4  00000+0 0  9996
2 25544 051.5908 168.3788 0125362 086.4185 359.7454 16.05064833    05


In [31]:
# 別の衛星の過去の全データ
data = st.gp_history(norad_cat_id=4, orderby='epoch', format='json')

In [32]:
with open('download/gp_history_4_all.json', 'w') as fp:
    fp.write(data)

In [33]:
df = pd.read_json(data)

In [34]:
# 古い衛星についてはOBJECT_NAME と TLE_LINE0 が TBA - TO BE ASSIGNED となってしまうことがある
df.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:35:20,18 SPCS,TBA - TO BE ASSIGNED,1958-001A,EARTH,TEME,UTC,SGP4,1959-07-09T22:15:15.874847,13.063418,0.115556,33.1849,192.1350,327.9492,35.6894,0,U,4,999,3800,0,0.000806,0,7615.485,110.231,2117.362,357.338,TBA,NaN,NaN,NaN,NaN,NaN,33905,8671417,0 TBA - TO BE ASSIGNED,1 00004U 58001 A 59190.92726707 .00080584 +00000-0 +00000-0 0 9997,2 00004 033.1849 192.1350 1155556 327.9492 035.6894 13.06341792038001
1,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:35:20,18 SPCS,TBA - TO BE ASSIGNED,1958-001A,EARTH,TEME,UTC,SGP4,1959-07-17T13:46:31.735488,13.075958,0.114989,33.1849,157.3663,20.6576,341.5910,0,U,4,999,3900,0,0.000830,0,7610.615,110.125,2107.621,357.339,TBA,NaN,NaN,NaN,NaN,NaN,33905,8671418,0 TBA - TO BE ASSIGNED,1 00004U 58001 A 59198.57397842 .00082991 +00000-0 +00000-0 0 9994,2 00004 033.1849 157.3663 1149895 020.6576 341.5910 13.07595833039007
2,2,GENERATED VIA SPACE-TRACK.ORG API,2004-08-15T23:35:20,18 SPCS,TBA - TO BE ASSIGNED,1958-001A,EARTH,TEME,UTC,SGP4,1959-08-01T20:45:25.598304,13.072853,0.115127,33.1849,87.4068,126.2272,239.3949,0,U,4,999,4100,0,0.000632,0,7611.820,110.151,2110.015,357.355,TBA,NaN,NaN,NaN,NaN,NaN,33905,8671419,0 TBA - TO BE ASSIGNED,1 00004U 58001 A 59213.86487961 .00063193 +00000-0 +00000-0 0 9999,2 00004 033.1849 087.4068 1151275 126.2272 239.3949 13.07285348041006


## 過去の特定の期間のデータを取得する

In [35]:
# EPOCHが特定の日の全データを取得 (creation_dateでもセレクション可)
data = st.gp_history(epoch=[op.inclusive_range('2020-01-01', '2020-01-02')], orderby=['norad_cat_id', 'epoch'], format='json')

In [36]:
with open('download/gp_history_2020-01-01.json', 'w') as fp:
    fp.write(data)

In [37]:
df = pd.read_json(data)

In [38]:
df.head(3)

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2020-01-01T06:30:17,18 SPCS,VANGUARD 1,1958-002B,EARTH,TEME,UTC,SGP4,2020-01-01T03:02:06.650304,10.847946,0.184560,34.2579,26.8849,314.8569,31.5834,0,U,5,999,18707,0.000245,0.000002,0.0,8619.925,132.744,3832.684,650.896,PAYLOAD,MEDIUM,US,1958-03-17,AFETR,NaN,2676442,145897261,0 VANGUARD 1,1 5U 58002B 20001.12646586 .00000199 00000-0 24469-3 0 9998,2 5 34.2579 26.8849 1845601 314.8569 31.5834 10.84794552187077
1,2,GENERATED VIA SPACE-TRACK.ORG API,2020-01-01T22:40:17,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-01-01T19:17:00.265056,11.856477,0.146792,32.8693,324.8532,25.0587,341.4531,0,U,11,999,59681,0.000153,0.000003,0.0,8123.902,121.452,2938.295,553.239,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,NaN,2676872,145926903,0 VANGUARD 2,1 11U 59001A 20001.80347529 .00000282 00000-0 15330-3 0 9995,2 11 32.8693 324.8532 1467925 25.0587 341.4531 11.85647749596815
2,2,GENERATED VIA SPACE-TRACK.ORG API,2020-01-02T06:13:51,18 SPCS,VANGUARD 2,1959-001A,EARTH,TEME,UTC,SGP4,2020-01-01T21:18:20.157984,11.856478,0.146792,32.8692,324.5369,25.5344,341.0943,0,U,11,999,25508,0.000155,0.000003,0.0,8123.902,121.452,2938.295,553.239,PAYLOAD,MEDIUM,US,1959-02-17,AFETR,NaN,2676929,145928075,0 VANGUARD 2,1 11U 59001A 20001.88773331 +.00000284 +00000-0 +15457-3 0 9999,2 11 032.8692 324.5369 1467925 025.5344 341.0943 11.85647808255085
